# Countdown Numbers Game

***

## Overview of Countdown Numbers Game

***

In [1]:
#Imports

# Permutations and combinations.
import itertools as it

# Random number generation.
import random

# Operators as functions.
import operator

In [2]:
# Generate Random Game Numbers
def new_numbers_game(no_large=None):
  # Returns six numbers and a target number representing a Countdown numbers game.
  
  # If no_large in None, randomly pick value between 0 and 4 inclusive.
  if no_large is None:
    # Randomly set the value.
    no_large = random.randrange(0, 5)
  
  # Select random large numbers.
  large_rand = random.sample([25, 50, 75, 100], no_large)
  # Select random small numbers.
  small_rand = random.sample(list(range(1, 11)) * 2, 6 - no_large)
  # The playing numbers.
  play_nos = large_rand + small_rand

  # Select a target number.
  target = random.randrange(101, 1000)

  # Return the game.
  return play_nos, target

In [3]:
# Random nubmers game.
gameNumbers = new_numbers_game()

In [4]:
gameNumbers[0]

[50, 75, 2, 4, 6, 2]

In [5]:
gameNumbers[1]

466

In [6]:
operators = [operator.add, operator.mul, operator.sub, operator.add, operator.add]

In [13]:
# # Operators.
# ops = [operator.add, operator.sub, operator.mul, operator.truediv]

# # All pair, op combs that hit target.
# for nos, op in it.product(it.permutations(gameNumbers[0], 6), ops):
#     #print(nos[0],nos[1],op(nos[0], nos[1]))
    
#     if op(nos[0], nos[1]) == 300:
#         print(nos[0], str(op), nos[1])

In [8]:
# Give all 2-partitions of a list
# where each sublist has  one element.
def patterns(numbers, operators):
  # Check if there is no way to partition further.
  if len(numbers) == 1:
    yield numbers
  # Loop through all the ways to partition L into two non-empty sublists.
  for i in range(1, len(numbers)):
    # Slice the list using i.
    for left, right in it.product(patterns(numbers[:i], operators[1:i]), patterns(numbers[i:], operators[i:])):
      # Yield the next operator applied to the sublists.
      yield [*left, *right, operators[0]]

In [9]:
# Evaluate RPN expression.
def eval_rpn(rpn):
  # A stack.
  stack = []
  # Loop through rpn an item at a time.
  for i in rpn:
    # Check if it's a number.
    if isinstance(i, int):
      # Append to the stack.
      stack = stack + [i]
    else:
      # Pop from stack twice.
      right = stack[-1]
      stack = stack[:-1]
      left = stack[-1]
      stack = stack[:-1]
      # Push operator applied to stack elements.
      stack = stack + [i(left, right)]
  # Should only be one item on stack.
  return stack[0]

In [10]:
numbers = [100, 75, 10, 4, 2, 1]

In [11]:
# Using eval, which mightn't be great.
for i in patterns(numbers, operators):
  print(eval_rpn(i), i)

325 [100, 75, 10, 4, 2, 1, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
325 [100, 75, 10, 4, 2, <built-in function add>, 1, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
925 [100, 75, 10, 4, <built-in function add>, 2, 1, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
1225 [100, 75, 10, 4, 2, <built-in function add>, <built-in function add>, 1, <built-in function sub>, <built-in function mul>, <built-in function add>]
1225 [100, 75, 10, 4, <built-in function add>, 2, <built-in function add>, 1, <built-in function sub>, <built-in function mul>, <built-in function add>]
555 [100, 75, 10, <built-in function sub>, 4, 2, 1, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function add>]
555 [100, 75, 10, <built-in function sub>, 4, 2, <built-in function add>, 1, <built-in

In [12]:
# Operators.
ops = [operator.add, operator.sub, operator.mul, operator.truediv]

# Limit the output.
limit = 1100

# For the limit.
i = 0
# Orderings of pairs.
for play_nos, opers in it.product(it.permutations(gameNumbers[0]), it.product(*([ops] * 5))):
  print(play_nos, opers)
  i = i + 1
  if i >= limit:
    break

(50, 75, 2, 4, 6, 2) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>)
(50, 75, 2, 4, 6, 2) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>)
(50, 75, 2, 4, 6, 2) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>)
(50, 75, 2, 4, 6, 2) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function truediv>)
(50, 75, 2, 4, 6, 2) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function add>)
(50, 75, 2, 4, 6, 2) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function sub>)
(50, 75, 2, 4, 6, 2) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in functio

## Complexity of Countdown Numbers Game

***

## Functional Aspect of Code behind Countdown Numbers Game

***

## References

***